In [683]:
#defining global values

# lcgrand module equivalent
from lcg_file import lcg #, LCG
from math import log

Q_LIMIT, BUSY, IDLE = 100, 1, 0

# integers
next_event_type, num_custs_delayed,\
    num_delays_required, num_events, \
        num_in_q, server_status = 0, 0, 0, 0, 0, 0

# floats
area_num_in_q, area_server_status, mean_interarrival, \
    mean_service, sim_time,time_last_event, \
        total_of_delays = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

# lists/arrays
time_arrival = []
time_next_event = []

In [684]:
def expon(mean):
    num = float(mean) * float(log(lcg.rand()))
    return -num


In [685]:
# read values from mm1.in and write base values to mm1.out
def write1(infile, outfile):
    global mean_interarrival, mean_service, num_delays_required
    with open(infile, 'r') as f:
        values = f.readline().strip().split()

        mean_interarrival, mean_service, num_delays_required = values

    with open(outfile, 'w') as f:
        f.write("Single-server queuing system\n\n")
        f.write(f'Mean interarrival time            {float(mean_interarrival):.3f} minutes\n\n')
        f.write(f'Mean service time                 {float(mean_service):.3f} minutes\n\n')
        f.write(f'Number of customers               {int(num_delays_required)}\n\n')

In [686]:
def initialize():
    

    # initialize state variables
    global server_status, num_in_q, time_last_event, num_custs_delayed,\
        total_of_delays, area_num_in_q, area_server_status, sim_time #, num_events

    sim_time = 0.0
    server_status = IDLE
    num_in_q = 0
    time_last_event = 0.0

    #initialize the statistical counters
    num_custs_delayed = 0
    
    total_of_delays = 0.0
    area_num_in_q  = 0.0
    area_server_status = 0.0

 
    # Initialize event list. Since no customers present, 
    # departure event is eliminated from consideration
    
    time_next_event.append(sim_time + expon(mean_interarrival) )
    time_next_event.append(1.0e+30)
    print(f'Initial event times array: {time_next_event}')

In [687]:
def timing(outfile):
    global next_event_type, sim_time, next_event_type

    min_time_next_event = 1.0e+29

    # determine the event type of the next event to occur
    for i in range(0, num_events):
        if (time_next_event[i] < min_time_next_event):
            min_time_next_event = time_next_event[i]
            next_event_type = i+1

    # check to see whether the event list is empty
    if (next_event_type == 0):
        with open(outfile, 'a') as f:
            f.write(f"\nEvent list is empty at time {sim_time}")
            exit(1)

    # event list not empty so advance simulation clock
    sim_time = min_time_next_event

In [688]:
def arrive(outfile):
    global server_status, total_of_delays, num_custs_delayed, time_next_event, num_in_q
    
    delay = 0.0

    time_next_event[0] = sim_time + expon(mean_interarrival)

    #check if server is busy
    if (server_status == BUSY):
        num_in_q += 1

        if (num_in_q > Q_LIMIT): # see if overflow condition exists
            with open(outfile, 'a') as f:
                # overflow occurred so stop simulation
                f.write(f"\nOverflow of the array time_arrival at time {sim_time:.1f}")
                exit(1)

        # there's room in the queue so store the time of arrival of arriving customer  
        time_arrival.append(sim_time)

    else:
        # serve idle so arriving customer has delay of zero
        delay = 0.0
        total_of_delays += delay

        # increment no of customers delayed, and make server busy
        num_custs_delayed += 1
        server_status = BUSY

        # schedule a departure (service completion)
        time_next_event[1] = sim_time + expon(mean_service)

In [689]:
def depart():
    global server_status, num_in_q, total_of_delays, num_custs_delayed, time_arrival

    print(f"\nInside depart: \n\
    Sim time: {sim_time}\n\
    total delays: {total_of_delays}\n\
    number in queue: {num_in_q}")

    delay = 0.0


    # check whether queue is empty
    if (num_in_q == 0):
        # queue empty so make server idle and eliminate the departure
        #  (service completion) event from consideration
        server_status = IDLE
        time_next_event[1] = 1.0e+30
        print(f"Time arrival array: {time_arrival}")
    
    else:
        # queue not empty, so decrement no of customers in queue     
        num_in_q -= 1

        print(f"time_arrival array: {time_arrival}")
        print(f"time_arrival[0]: {time_arrival[0]}")

        # compute delay of the custome beginning service
        # and update total delay accumulator
        delay = sim_time - time_arrival[0]
        total_of_delays += delay

        print(f"Delay: {delay}\n\
        New total of delays: {total_of_delays}")

        # move each customer in queue (if any) up one place
        time_arrival.pop(0)

    
        # increment no of customers delayed and schedule departure
        num_custs_delayed += 1
        time_next_event[1] = sim_time + expon(mean_service)

In [690]:
def report(outfile):
    with open(outfile, 'a') as f:
        f.write(f"\n\nAverage delay in queue            {(total_of_delays / num_custs_delayed):.3f} minutes \n\n")
        f.write(f"Average number in queue           {(area_num_in_q / sim_time):.3f}\n\n")
        f.write(f"Server utilization                {(area_server_status / sim_time):.3f}\n\n")
        f.write(f"Time simulation ended             {(sim_time):.3f} minutes")

In [691]:
def update_time_avg_stats():
    global time_last_event, area_num_in_q, area_server_status

    # compute time since last event and update last_event_time marker
    time_since_last_event = sim_time - time_last_event
    time_last_event = sim_time

    # update area under number_in_queue function
    area_num_in_q += (num_in_q * time_since_last_event)

    # update area under server-busy indicator function
    area_server_status += (server_status * time_since_last_event)

In [692]:
def main():
    global num_events
    
    num_events = 2 # no of events for the timing function

    input_file = "mm1.in"
    output_file = "mm1.out"

    write1(input_file, output_file) # read from infile and write to outfile

    initialize() # initialize the simulation

    while(int(num_custs_delayed) < int(num_delays_required)):
        timing(output_file) # determine next event

        update_time_avg_stats() # update time average statistical accumulators

        if (next_event_type == 1): # invoke appropriate event according to next_event_type value
            arrive(output_file)
            # break
        else:
            depart()
            # break

    report(output_file) # write the statistics observed to outfile

main()

Initial event times array: [1.2431673689219338, 1e+30]

Inside depart: 
    Sim time: 1.497747224188611
    total delays: 0.0
    number in queue: 0
Time arrival array: []

Inside depart: 
    Sim time: 3.1078218543634115
    total delays: 0.0
    number in queue: 0
Time arrival array: []

Inside depart: 
    Sim time: 6.935401826548478
    total delays: 0.0
    number in queue: 0
Time arrival array: []

Inside depart: 
    Sim time: 9.61060937896151
    total delays: 0.0
    number in queue: 1
time_arrival array: [9.57125078356257]
time_arrival[0]: 9.57125078356257
Delay: 0.039358595398939045
        New total of delays: 0.039358595398939045

Inside depart: 
    Sim time: 10.797087943833994
    total delays: 0.039358595398939045
    number in queue: 1
time_arrival array: [10.406667100503423]
time_arrival[0]: 10.406667100503423
Delay: 0.3904208433305705
        New total of delays: 0.4297794387295095

Inside depart: 
    Sim time: 10.889772907407057
    total delays: 0.4297794387295095